# Project Euler Problem 118:<br><i>Riffle shuffles</i>

Link to original problem prompt: [https://projecteuler.net/problem=622](https://projecteuler.net/problem=622)

## Approach

My approach was to first to visualize how a given deck changes after a shuffle and hopefully recognize a pattern. This can be done effortlessly via a function as follows:

```
import numpy as np

# `deck` is a NumPy array denoting the ordered deck of cards
# `n` is the total number of cards
# specifically, original `deck` = np.arange(1, n + 1)
def riffle_shuffle(deck, n):
    print('Riffle-shuffling on:')
    print(deck)

    left = deck[: n // 2]
    right = deck[n // 2 :]
    inter = np.ravel(np.column_stack((left, right)))

    print('Left-half:')
    print(left)
    print('Right-half:')
    print(right)
    print('Resulting interleaved deck:')
    print(inter)
```

Recognizing a pattern is fairly easy after using the above function with some random $n$. For example, one can hypothesize that the difference between any pair of adjacent cards in the deck is the same across all pairs of adjacent cards in module $n - 1$ (this difference might, however, change after each shuffle), and utilize mathematical induction to prove it.

A corollary of that is that as soon as the original second card (the second card in the deck before any shuffling) returns to the second slot in the deck, every other card will also be at their original location. This means that we only have to count the number of shuffles it would take for the original second card to return to its original slot.

## Analyzing changes in the location of the original second card

If we consider the series ${x_i}$ to denote the distance of the original second card in the deck to the first card in the deck (which is fixed) after the $i$th shuffle (with $i$ is greater than or equal to 1), the function $s(n)$ from the question prompt will be to determine the smallest $k$ such that $x_k = 1$ (when the original second card is right next to the first). Furthermore, the change from $x_i$ to $x_{i + 1}$ (from one element in the series to the next) indicates the change in location of the card. We also have:

- $x_0 = 1$ (the second card is next to the first before any shuffling)
- $x_{i + 1} = 2x_i$ if $2x_i < n$
- $x_{i + 1} = 2x_i - n + 1$ if $2x_i >= n$

The above can be proved, again, via mathematical induction. We can also notice that the last two equations is equivalent to $x_{i + 1} = 2x_i\ mod\ (n - 1)$ (notice that this is true only because $n$ is always an even number). This will translate to a nice formula for the elements in the series: $x_i = 2^i\ mod\ (n - 1)$.

Therefore, for a given $n$, function $s(n)$ yields the smallest $k$ such that $2^k = 1\ mod\ (n - 1)$. However, as for the question of finding all values of $n$ such that $s(n) = 60$, we would need to do some reverse engineering.

## Finding all values of $n$ satisfying the given condition

The question now is to find all $n$'s such that the smallest solution for $k$ in the equation $2^k = 1\ mod\ (n-1)$ is 60. This means that all of those $n$'s satisfy $2^{60} = 1\ mod\ (n-1)$. The problem is now simple: to find all factors of $2^{60} - 1$ and check back to see if a given factor $a$ satisfies the condition that $s(a) = 60$. To do this, I wrote a function to generate all factors (divisors) of a number from its prime factorization:

```
# `coprime_divisors` is a list of tuple (prime_factor, power)
# denoting in the prime factorization of a number
def generate_divisors(coprime_divisors):
    running_divisors = [1]

    for coprime_divisor, power in coprime_divisors:
        new_divisors = []
        for p in range(1, power + 1):
            for divisor in running_divisors:
                new_divisors.append(divisor * (coprime_divisor ** p))

        running_divisors += new_divisors

    return running_divisors
```

Finally, I wrote my own $s(n)$ function to filter out all the valid numbers from the list of factors of $2^{60} - 1$.

```
def s(n):
    def get_next_two_index(temp, n):
        double = 2 * temp

        if double < n:
            return double

        return double - n + 1

    count = 0
    two_index = 1
    while True:
        two_index = get_next_two_index(two_index, n)
        count += 1
        if two_index == 1:
            break

    return count
```

Notice that this function follows the same logic regarding the location of the original second card described above.

## Conclusions

The pressure was definitely on with this one as a lot of people were able to solve it in a short amount of time, giving it a difficulty level of 15% (at the time of this writing). Luckily I was able to find my own solution relatively quickly too.